traj2grid测试

In [2]:
import torch
file_name = "model/cp_139_loss0.273_rank_12.47.pth"
model = torch.load(file_name)


AttributeError: 'dict' object has no attribute 'to'

In [31]:
from traj2grid import Traj2Grid
from parameters import min_lon, max_lon, min_lat, max_lat
import json
str_grid2idx = json.load(open("data/str_grid2idx_400_44612.json"))
grid2idx = {eval(g): str_grid2idx[g] for g in list(str_grid2idx)}
t2g = Traj2Grid(400, 400, min_lon, min_lat, max_lon, max_lat, grid2idx)
model.t2g = t2g
model.mean_x = 104.08007913272864
model.mean_y = 30.68234119864428
model.std_x = 0.021409964446530056
model.std_y = 0.019668597464374094

In [32]:
torch.save(model, f"model/archived_model/{model_name}")